In [257]:
import re
import json
from datetime import datetime
from collections import defaultdict

In [419]:
import os
from pathlib import Path

PROJECT_ROOT = Path(os.path.abspath('')).parent
print(f"{PROJECT_ROOT=}")

log_file = "/Users/tug/Downloads/leo_mic (4).log"
transalation_file = "/Users/tug/Work/projects/neuralstream/synchro-python/sources/4.txt"
config_file = "/Users/tug/Work/projects/neuralstream/synchro-python/config.local.json"

report_raw_file = "/Users/tug/Downloads/report.json"
report_html_file = "/Users/tug/Downloads/report.html"

PROJECT_ROOT=PosixPath('/Users/tug/Work/projects/neuralstream/synchro-python')


In [271]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize


def calculate_bleu(reference_text, candidate_text):
    """
    Подробнее в bleu.ipynb
    """
    # Токенизация
    reference_tokens = [word_tokenize(reference_text)]
    candidate_tokens = word_tokenize(candidate_text)

    # Расчёт BLEU
    smoothing = SmoothingFunction().method1
    return sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothing)

In [291]:
from translate import Translator


def translate(text: str, source: str = "en", target: str = "ru") -> str:
    # Кешируем инстанс переводчика
    translator = getattr(translate, "instance", None)
    if translator is None:
        translator = Translator(from_lang=source, to_lang=target)
    else:
        setattr(translate, "instance", translator)

    # Переводим текст, если получается, иначе оставляем как есть
    try:
        result = translator.translate(text)
    except RuntimeError:
        result = text
        
    return result

In [400]:
# mermaid graph

def get_graph(edges):
    # Генерация Mermaid-кода
    result = f"graph TD\n"
    get_params = lambda x: " : ".join([
        str(x) 
        for x in (
            edge_from.get("device"), 
            edge_from.get("stream", {}).get("rate")
        )
        if x is not None
    ])
    for edge_from, edge_to in edges:
        result += f'  {edge_from["name"]}[{edge_from["name"]}<br>{get_params(edge_from)}]\n'
        result += f'  {edge_to["name"]}[{edge_to["name"]}<br>{get_params(edge_to)}]\n'
        
        
    for edge_from, edge_to in edges:
        result += f'  {edge_from["name"]} --> {edge_to["name"]}\n'

    return f"{result}"

sample = [({'name': 'input_mic_0_en', 'node_type': 'input_channel', 'device': 0, 'channel': 1, 'stream': {'language': 'en', 'rate': 44100, 'audio_format': {'format_type': 'int16'}}}, {'name': 'resampler_input_0_en', 'node_type': 'resampler', 'to_rate': 16000}), ({'name': 'resampler_output_1', 'node_type': 'resampler', 'to_rate': 44100}, {'name': 'output_device_1_ru', 'device': 1, 'channel': 1, 'stream': {'language': 'ru', 'audio_format': {'format_type': 'int16'}, 'rate': 44100}, 'node_type': 'output_channel'}), ({'name': 'resampler_output_1', 'node_type': 'resampler', 'to_rate': 44100}, {'name': 'output_file_1_ru', 'node_type': 'output_file', 'path': 'samples/translated_output.wav', 'stream': {'language': 'ru', 'audio_format': {'format_type': 'int16'}, 'rate': 44100}}), ({'name': 'mixer_1_ru', 'node_type': 'mixer'}, {'name': 'resampler_output_1', 'node_type': 'resampler', 'to_rate': 44100}), ({'name': 'mixer_0_ru'}, {'name': 'output_file_0_ru'}), ({'name': 'converter_0_en_ru', 'node_type': 'converter_seamless', 'server_url': 'https://09f1-91-73-240-61.ngrok-free.app', 'log_file': 'session.log', 'config': {'stt': {'lang': 'en', 'stop_words': ['DimaTorzok', 'субтитры сделал', 'субтитры делал', 'M.K.'], 'buffer_min_words_size': 6, 'buffer_timeout_seconds': 1.0}, 'translate': {'lang_from': 'en', 'lang_to': 'ru', 'llm_call_timeout': 15.0, 'context_window': 4, 'forced_line_end': '...', 'escaped_characters': ['\n', '\r', '\t', '...'], 'language_map': {'ru': 'Russian', 'en': 'English'}, 'text_template': 'I want you to act as a skilled translator.\nI will provide you with:\n- the original context (previous parts in the source language),\n- the translated context (previous parts translated into the target language),\n- the source and target languages,\n- the next part to translate (new input in the source language).\n\nYour task is to:\n1. Translate the "next part to translate" into the target language, without any additional explanations or comments, only the translated phrase.\n2. Ensure that the translation is consistent with the "original context" and continues seamlessly from the "translated context".\n3. If there are words or phrases in the original context that are missing in the translated context, do not retranslate them. Focus only on translating the "next part to translate".\n4. Provide the translation as a direct continuation of the translated context.\n\nOriginal context (source language): {source_context}\nTranslated context (target language): {translated_context}\nSource language: {lang_from}\nTarget language: {lang_to}\nNext part to translate (source language): {text}', 'correction_template': ''}, 'tts': {'lang': 'ru', 'gap_size_bytes': 12000, 'voice_map': {'en': ['piper', 'hfc_male'], 'ru': ['piper', 'ruslan']}}}}, {'name': 'mixer_1_ru', 'node_type': 'mixer'}), ({'name': 'resampler_input_0_en', 'node_type': 'resampler', 'to_rate': 16000}, {'name': 'converter_0_en_ru', 'node_type': 'converter_seamless', 'server_url': 'https://09f1-91-73-240-61.ngrok-free.app', 'log_file': 'session.log', 'config': {'stt': {'lang': 'en', 'stop_words': ['DimaTorzok', 'субтитры сделал', 'субтитры делал', 'M.K.'], 'buffer_min_words_size': 6, 'buffer_timeout_seconds': 1.0}, 'translate': {'lang_from': 'en', 'lang_to': 'ru', 'llm_call_timeout': 15.0, 'context_window': 4, 'forced_line_end': '...', 'escaped_characters': ['\n', '\r', '\t', '...'], 'language_map': {'ru': 'Russian', 'en': 'English'}, 'text_template': 'I want you to act as a skilled translator.\nI will provide you with:\n- the original context (previous parts in the source language),\n- the translated context (previous parts translated into the target language),\n- the source and target languages,\n- the next part to translate (new input in the source language).\n\nYour task is to:\n1. Translate the "next part to translate" into the target language, without any additional explanations or comments, only the translated phrase.\n2. Ensure that the translation is consistent with the "original context" and continues seamlessly from the "translated context".\n3. If there are words or phrases in the original context that are missing in the translated context, do not retranslate them. Focus only on translating the "next part to translate".\n4. Provide the translation as a direct continuation of the translated context.\n\nOriginal context (source language): {source_context}\nTranslated context (target language): {translated_context}\nSource language: {lang_from}\nTarget language: {lang_to}\nNext part to translate (source language): {text}', 'correction_template': ''}, 'tts': {'lang': 'ru', 'gap_size_bytes': 12000, 'voice_map': {'en': ['piper', 'hfc_male'], 'ru': ['piper', 'ruslan']}}}}), ({'name': 'input_mic_0_en', 'node_type': 'input_channel', 'device': 0, 'channel': 1, 'stream': {'language': 'en', 'rate': 44100, 'audio_format': {'format_type': 'int16'}}}, {'name': 'output_file_mic', 'node_type': 'output_file', 'path': 'samples/source_input.wav', 'stream': {'language': 'ru', 'audio_format': {'format_type': 'int16'}, 'rate': 44100}})]
print(get_graph(sample))

graph TD
  input_mic_0_en[input_mic_0_en<br>0 : 44100]
  resampler_input_0_en[resampler_input_0_en<br>0 : 44100]
  resampler_output_1[resampler_output_1<br>]
  output_device_1_ru[output_device_1_ru<br>]
  resampler_output_1[resampler_output_1<br>]
  output_file_1_ru[output_file_1_ru<br>]
  mixer_1_ru[mixer_1_ru<br>]
  resampler_output_1[resampler_output_1<br>]
  mixer_0_ru[mixer_0_ru<br>]
  output_file_0_ru[output_file_0_ru<br>]
  converter_0_en_ru[converter_0_en_ru<br>]
  mixer_1_ru[mixer_1_ru<br>]
  resampler_input_0_en[resampler_input_0_en<br>]
  converter_0_en_ru[converter_0_en_ru<br>]
  input_mic_0_en[input_mic_0_en<br>0 : 44100]
  output_file_mic[output_file_mic<br>0 : 44100]
  input_mic_0_en --> resampler_input_0_en
  resampler_output_1 --> output_device_1_ru
  resampler_output_1 --> output_file_1_ru
  mixer_1_ru --> resampler_output_1
  mixer_0_ru --> output_file_0_ru
  converter_0_en_ru --> mixer_1_ru
  resampler_input_0_en --> converter_0_en_ru
  input_mic_0_en --> output_fil

In [ ]:
# analyze_audio

import librosa
import numpy as np
from scipy.signal import resample
from pysepm import pesq, stoi


def analyze_audio(file_path, reference_path=None, sr_target=16000):
    """
    Анализирует качество аудио на основе объективных метрик.
    :param file_path: Путь к аудиофайлу для анализа.
    :param reference_path: Путь к эталонному аудио (если доступно).
    :param sr_target: Частота дискретизации для анализа (по умолчанию 16 кГц для PESQ).
    :return: Словарь с метриками качества.
    """
    # Загружаем аудио
    y, sr = librosa.load(file_path, sr=None)
    print(f"Загружен файл: {file_path}, длина {len(y) / sr:.2f} сек, частота {sr} Гц")
    
    # Если есть эталонное аудио, загружаем его
    ref_y = None
    if reference_path:
        ref_y, ref_sr = librosa.load(reference_path, sr=None)
        # Приводим эталонное аудио к той же длине, что и тестовое
        if len(ref_y) > len(y):
            ref_y = ref_y[:len(y)]
        elif len(ref_y) < len(y):
            y = y[:len(ref_y)]
    
    # Пересэмплинг для анализа
    y_resampled = librosa.resample(y, orig_sr=sr, target_sr=sr_target)
    if ref_y is not None:
        ref_y_resampled = librosa.resample(ref_y, orig_sr=ref_sr, target_sr=sr_target)
    else:
        ref_y_resampled = None

    # Инициализация метрик
    metrics = {}

    # 1. SNR (Signal-to-Noise Ratio)
    signal_power = np.mean(y_resampled**2)
    noise_power = np.var(y_resampled - ref_y_resampled) if ref_y_resampled is not None else None
    metrics['SNR (dB)'] = 10 * np.log10(signal_power / noise_power) if noise_power else None

    # 2. PESQ (если есть эталонное аудио)
    if ref_y_resampled is not None:
        metrics['PESQ'] = pesq(ref_y_resampled, y_resampled, sr_target)

    # 3. STOI (если есть эталонное аудио)
    if ref_y_resampled is not None:
        metrics['STOI'] = stoi(ref_y_resampled, y_resampled, sr_target)

    # 4. Spectral Centroid (средняя частота спектра)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    metrics['Average Spectral Centroid (Hz)'] = np.mean(spectral_centroid)

    # 5. Zero Crossing Rate (Частота пересечения нуля)
    zcr = librosa.feature.zero_crossing_rate(y=y)
    metrics['Zero Crossing Rate'] = np.mean(zcr)

    # 6. RMS Energy (корень среднего квадрата)
    rms = librosa.feature.rms(y=y)
    metrics['Average RMS Energy'] = np.mean(rms)

    return metrics

In [ ]:
# log handlers

def fdt(dt):
    return dt.strftime("%H:%M:%S.%f")[:-3]

def audio_bytes_handler(match, metrics, dt):
    metrics["total_audio_bytes"] += int(match.group(1))

def task_completed_handler(match, metrics, dt):
    metrics["total_tasks"] += 1

def stt_handler(match, metrics, dt):
    value = match.group(1)

    metrics["texts"]["stt"].append(value)

    metrics["buffer"].append((dt, "stt", value))

def stti_handler(match, metrics, dt):
    value = match.group(1)

    metrics["texts"]["stti"].append(value)

    metrics["buffer"].append((dt, "stti", value))

def llm_handler(match, metrics, dt):
    value = match.group(1)

    metrics["texts"]["llm"].append(value)

    metrics["buffer"].append((dt, "llm", value))

def tts_handler(match, metrics, dt):
    value = match

    duration = (metrics["buffer"][-1][0] - metrics["buffer"][0][0]).total_seconds()
    metrics["processing_time"].append(duration)

    metrics["texts"]["tts"].append(value)

    metrics["buffer"].append((dt, "tts", value))
    metrics["chunks"].append(metrics["buffer"])
    metrics["buffer"] = []

In [293]:
PATTERNS_MAP = {
    r"Received audio bytes (\d+)": audio_bytes_handler,
    r"Yielding item (\d+)": task_completed_handler,
    r"root\.\[STT\]: [^ ]+ Received response  (.+)": stt_handler,
    r"root\.\[STT\]: [^ ]+ Received transcription: (.+)": stti_handler,
    r"root\.\[LLM\]: [^ ]+ Sent translation: (.+)": llm_handler,
    r"root\.\[TTS\]: [^ ]+ Synthesizing text using .+": tts_handler,
}

In [379]:
# parse_log

def parse_log(log_file_path):
    """
    Parses a log file and extracts key metrics for report generation.
    :param log_file_path: Path to the log file.
    :return: Dictionary containing aggregated metrics.
    """
    # Инициализация метрик
    metrics = {
        "total_audio_bytes": 0,
        "total_tasks": 0,
        "processing_time": [],
        "errors": [],
        "buffer": [],
        "chunks": [],
        "texts": defaultdict(list),
    }

    def process_line(line, metrics):
        handler, params = getattr(process_line, "future", None) or [None, {}]
        if callable(handler):
            handler(line, metrics, **params)
            setattr(process_line, "future", None)
            return

        for pattern, handler in PATTERNS_MAP.items():
            match = re.search(pattern, line)
            if not match:
                continue
            
            dt = datetime.fromisoformat(line[:26])
            if handler == tts_handler:
                setattr(process_line, "future", (handler, {"dt": dt}))
            else:
                handler(match, metrics, dt)

            return

    with open(log_file_path, "r") as fp:
        [process_line(x, metrics) for x in fp]
    
    return metrics

In [ ]:
# generate_html_report
from jinja2 import Template

def generate_html_report(report_data, output_file):
    """
    Генерирует HTML-страницу отчёта с использованием Jinja2 и Bootstrap.
    
    :param report_data: Словарь с данными отчёта
    :param output_file: Путь до выходного HTML-файла
    """
    # HTML-шаблон с использованием Bootstrap
    with open("report.html", "r") as fp:
        html_template = fp.read()

    # Создание шаблона
    template = Template(html_template)

    # Генерация HTML-кода
    html_content = template.render(report_data=report_data)

    # Запись HTML в файл
    with open(output_file, "w") as file:
        file.write(html_content)

    print(f"Отчёт сохранён в файл: {output_file}")


In [422]:
# generate_report

def generate_report(metrics, output_file):
     # Конфиг
    with open(config_file, "r") as fp:
        config = json.load(fp)

    nodes = {
        x["name"]: x 
        for x in config["nodes"]
    }

    config_source_output = nodes.get("output_file_mic")
    audio_source_file = str(PROJECT_ROOT / config_source_output.get("path"))

    key = "output_file_1_ru"
    config_result_output = nodes.get(key)
    audio_output_file = str(PROJECT_ROOT / config_result_output.get("path"))
    
    key = "converter_0_en_ru"
    config_converter = nodes.get(key)

    # Граф узлов
    edges = [
        (nodes.get(k1, {"name": k1}), nodes.get(k2, {"name": k2})) 
        for k1, k2 in config["edges"]
    ]

    graph = get_graph(edges)
    
    # Агрегируем среднее время обработки
    average_time = int((
        sum(metrics["processing_time"]) / len(metrics["processing_time"]) 
        if metrics["processing_time"] else 
        0
    ) * 1000)
    
    # Готовим чанки
    chunks = metrics["chunks"]

    # Форматируем время для удобного чтения
    chunks = [
        [(fdt(x[0]), x[1], x[2]) for x in block]
        for block in chunks
    ]

    # Собираем все чанки
    chunks = [
        [";".join(x) for x in block] 
        for block in chunks
    ]

    # Собираем полные тексты
    texts = {
        k: " ".join([x.strip() for x in v]) 
        for k, v in metrics["texts"].items()
    }

    # Считаем BLEU для текста переведенного по полному тексту от TTS
    if "ref_stt" not in texts:
        texts["ref_stt"] = translate(
            texts["stti"][:500],
            config_converter["config"]["translate"]["lang_from"],
            config_converter["config"]["translate"]["lang_to"],
        )
    bleu_stt = calculate_bleu(texts["ref_stt"], texts["ref_stt"])

    # Считаем BLEU для эталонного перевода
    bleu_ref = "No reference translation provided"
    if transalation_file:
        with open(transalation_file, "r") as fp:
            texts["ref"] = fp.read()
            
        bleu_ref = calculate_bleu(texts["ref"], texts["tts"])

    bleu_ref_ref_stt = calculate_bleu(texts["ref"], texts["ref_stt"])
    
    # Формируем финальный отчёт
    report = {
        "full_config": config,
        "converter_config": json.dumps(config_converter, indent=4, ensure_ascii=False),
        "graph": graph,
        "source_audio": audio_source_file,
        "result_audio": audio_output_file,
        "bytes_total": metrics["total_audio_bytes"],
        "tasks_total": metrics["total_tasks"],
        "processing_time_avg": average_time,
        "chunks": chunks,
        "texts": texts,
        "bleu": [
            ["Перевод", "На озвучке", bleu_ref],
            ["Автоматический перевод 500 символов", "На озвучке", bleu_stt],
            ["Перевод", "Автоматический перевод 500 символов", bleu_ref_ref_stt],
        ],
        "created_at": datetime.now().isoformat(),
    }
    
    # Сохраняем отчёт в JSON
    with open(output_file, "w") as file:
        json.dump(report, file, indent=4, ensure_ascii=False)

    print(f"Report saved to {output_file}")

    return report

In [423]:
# Пути к лог-файлу и отчёту
print("Original log from", log_file)

# Парсим лог и создаём отчёт
metrics = parse_log(log_file)
report_data = generate_report(metrics, report_raw_file)
generate_html_report(report_data, report_html_file)
print("Done")

Original log from /Users/tug/Downloads/leo_mic (4).log
Report saved to /Users/tug/Downloads/report.json
Отчёт сохранён в файл: /Users/tug/Downloads/report.html
Done
